In [1]:
from load_models import load_ckpt, get_dataloader

In [2]:
single_ckpt_path = './runs/0504out_dim5proj_dim5batch-size512lr0.0003epochs800fp16use_gptis_causaln_embd32add_train/checkpoint.pth'
multi_ckpt_path = '/moto/stats/users/ch3676/spike_ddp_models_0504/0502_mc_gpt_conseq_causal_nembd64_block1331_bs128_extra5_lr0.0001_knn10_addtrain/checkpoint.pth'
multi_data_path='/home/gridsan/cloh/spike_data/multi_dy016_random_neurons_04_28_2023'
single_data_path='/home/gridsan/cloh/spike_data/single_dy016_random_neurons_04_28_2023'


In [3]:
model = load_ckpt(multi_ckpt_path, multi_chan = True, rep_after_proj=True)

number of parameters: 0.99M
Using projector; batchnorm False with depth 3; hidden_dim=512
missing keys []
unexpected keys ['online_head.weight', 'online_head.bias']


In [4]:
train_loader = get_dataloader(multi_data_path, multi_chan=True, split='train')
test_loader = get_dataloader(multi_data_path, multi_chan=True, split='test')


True spikes_train.npy
(12000, 21, 121)
(3000, 21, 121)


In [5]:
from ddp_utils import knn_predict
import torch
def knn_monitor(net, memory_data_loader, test_data_loader, device='cuda', k=200, t=0.1, hide_progress=False,
                targets=None, multi_chan=False):
    if not targets:
        targets = memory_data_loader.dataset.targets

    net.eval()
    classes = 100
    total_top1, total_top5, total_num, feature_bank = 0.0, 0.0, 0, []
    with torch.no_grad():
        for data, target in memory_data_loader:
            if not multi_chan:
                data = torch.squeeze(data, dim=1)
                data = torch.unsqueeze(data, dim=-1)
            else:
                data = data.view(-1, 11*121)
                data = torch.unsqueeze(data, dim=-1)
            
            feature = net(data.to(device=device, non_blocking=True))
            
            feature = torch.nn.functional.normalize(feature, dim=1)
            feature_bank.append(feature)
        # [D, N]
        feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
        # [N]
        feature_labels = torch.tensor(targets, device=feature_bank.device)
        
        # loop test data to predict the label by weighted knn search
        for data, target in test_data_loader:
            
            data, target = data.to(device=device, non_blocking=True), target.to(device=device, non_blocking=True)
            
            if not multi_chan:
                data = torch.squeeze(data, dim=1)
                data = torch.unsqueeze(data, dim=-1)
            else:
                data = data.view(-1, 11*121)
                data = torch.unsqueeze(data, dim=-1)
            
            feature = net(data)
            
            feature = torch.nn.functional.normalize(feature, dim=1)

            pred_labels = knn_predict(feature, feature_bank, feature_labels, classes, k, t)

            total_num += data.size(0)
            total_top1 += (pred_labels[:, 0] == target).float().sum().item()
    return total_top1 / total_num * 100

In [8]:
knn_score = knn_monitor(net=model.cuda(), memory_data_loader=train_loader, test_data_loader=test_loader, device='cuda',k=200, hide_progress=True, multi_chan=True)
print("knn if rep AFTER proj:", knn_score)

knn if rep AFTER proj: 98.56666666666666


In [9]:
model2 = load_ckpt(multi_ckpt_path, multi_chan = True, rep_after_proj=False)
knn_score = knn_monitor(net=model2.cuda(), memory_data_loader=train_loader, test_data_loader=test_loader, device='cuda',k=200, hide_progress=True, multi_chan=True)
print("knn if rep BEFORE proj:", knn_score)

number of parameters: 0.99M
missing keys []
unexpected keys ['projector.proj_block.0.weight', 'projector.proj_block.0.bias', 'projector.proj_block.2.weight', 'projector.proj_block.2.bias', 'projector.proj_block.4.weight', 'projector.proj_block.4.bias', 'online_head.weight', 'online_head.bias']
knn if rep BEFORE proj: 98.33333333333333


In [3]:
## Single channel

model = load_ckpt(single_ckpt_path, multi_chan = False, rep_after_proj=False)
single_train_loader = get_dataloader(single_data_path, multi_chan=False, split='train')
single_test_loader = get_dataloader(single_data_path, multi_chan=False, split='test')


Using projector; batchnorm False with depth 3; hidden_dim=512
number of parameters: 0.52M
missing keys ['backbone.projector.proj_block.0.weight', 'backbone.projector.proj_block.0.bias', 'backbone.projector.proj_block.2.weight', 'backbone.projector.proj_block.2.bias', 'backbone.projector.proj_block.4.weight', 'backbone.projector.proj_block.4.bias']
unexpected keys ['projector.proj_block.0.weight', 'projector.proj_block.0.bias', 'projector.proj_block.2.weight', 'projector.proj_block.2.bias', 'projector.proj_block.4.weight', 'projector.proj_block.4.bias']
False spikes_train.npy
(12000, 121)
(3000, 121)


In [6]:
knn_score = knn_monitor(net=model.cuda(), memory_data_loader=single_train_loader, test_data_loader=single_test_loader, device='cuda',k=200, hide_progress=True, multi_chan=False)
print("Single channel; knn rep BEFORE proj:", knn_score)

Single channel; knn rep BEFORE proj: 83.73333333333333


In [7]:
model2 = load_ckpt(single_ckpt_path, multi_chan = False, rep_after_proj=True)


Using projector; batchnorm False with depth 3; hidden_dim=512
number of parameters: 0.52M
Using projector; batchnorm False with depth 3; hidden_dim=512
missing keys ['backbone.projector.proj_block.0.weight', 'backbone.projector.proj_block.0.bias', 'backbone.projector.proj_block.2.weight', 'backbone.projector.proj_block.2.bias', 'backbone.projector.proj_block.4.weight', 'backbone.projector.proj_block.4.bias']
unexpected keys []


In [9]:
knn_score = knn_monitor(net=model2.cuda(), memory_data_loader=single_train_loader, test_data_loader=single_test_loader, device='cuda',k=200, hide_progress=True, multi_chan=False)
print("Single channel; knn rep AFTER proj:", knn_score)

Single channel; knn rep AFTER proj: 73.66666666666667
